Cleaning part only. Further analysis conducted in Stata

In [5]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt



Cleaning part 1: create new song ID with instances and flag first and last

In [6]:
pd.set_option('display.max_columns', None)

songs = pd.read_csv('Hot Stuff.csv')

#turn weekID to date
songs.WeekID = pd.to_datetime(songs.WeekID)

#print(songs.info())
songs_trunc = songs.sort_values(by = ['SongID', 'Instance', 'Weeks on Chart'])

#creating songid_2
new = songs_trunc['Instance'].astype(str).copy()
#print (songs_trunc.info())
songs_trunc['SongID_2'] = songs_trunc['SongID'].str.cat(new, sep = " ")
#print(songs_trunc['SongID_2'])

#creating flag for first and last occurence (takes into account instances)
songs_trunc['flag'] = ((songs_trunc.SongID_2 != songs_trunc.SongID_2.shift()) |
                                             (songs_trunc.SongID_2 != songs_trunc.SongID_2.shift(-1))).astype(int)
#only save flag = 1 to a csv
songs_cleanish = songs_trunc.loc[songs_trunc.flag == 1, :]
songs_cleanish.to_csv('songs_cleanish_2.csv')


'''Testing: 
songs_clean = songs.sort_values('SongID', 'Instance', 'Weeks On Chart')
songs_sort_sub = songs_sort.iloc[320000:320494,:]
#songs_sort_sub.to_csv('last subset.csv')
week1228 = songs_sort.loc[songs_sort.WeekID == datetime(2019,12,28), :]
#print(week1228)
week1221 = songs_sort.loc[songs_sort.WeekID == datetime(2019,12,21),:]
#print(week1221)
#print(songs_sort.iloc[320400:320494,:])'''

KeyboardInterrupt: 

Cleaning part 2: create debut_week in accordane for each row to be used for survival analysis

In [ ]:
#checkpoint 1
songs = pd.read_csv('songs_cleanish_2.csv')

print(songs.info())
print(songs.describe())

#flag the last row for every instance
songs['flag_last'] = ((songs.SongID_2 != songs.SongID_2.shift(-1))).astype(int)


#Create debut_week and debut_position conditions
# ['WeekID'] of flag = 0 --> ['Debut Week'] for index + 1
# ['Week Postion'] of flag = 0 --> ['Debut Position'] for index + 1

cond1 = songs['flag_last'] == songs['flag']
cond2 = songs['flag_last'] != songs['flag_last'].shift()
cond3 = songs['flag_last'] == songs['flag_last'].shift()


#Make debut_week and debut_position
songs['Debut_Week_2'] = np.where((cond1) & (cond2),
            songs['WeekID'].shift(),
            np.NaN)
songs['Debut_Week_3'] = np.where(cond3,
            songs['WeekID'],
            np.NaN)
songs['Debut_Position_2'] = np.where((cond1) & (cond2),
            songs['Week Position'].shift(),
            np.NaN)
songs['Debut_Position_3'] = np.where(cond3,
            songs['Week Position'],
            np.NaN)

'''Testing: 
songs['flag_temp_check'] = ((songs.Debut_Week == songs.Debut_Week.shift())).astype(int)
print(songs.flag_temp_check.sum())'''

print(songs)

songs.to_csv('songs_cleanish_4.csv')


Cleaning part 3: consolidate debut_week, take care of right censoring, create month and year for later manipulation

In [ ]:
#checkpoint 2
songs = pd.read_csv('songs_cleanish_4.csv')

#subset to only last distinct entry
songs_sub = songs.loc[songs['flag_last'] == 1, :]
print(songs_sub.info())

#combining columns for debut_week and debut_position
songs_sub['Debut_Week_2'].fillna(songs_sub['Debut_Week_3'], inplace = True)
songs_sub['Debut_Position_2'].fillna(songs_sub['Debut_Position_3'], inplace = True)

songs_sub.drop(columns = ['Previous Week Position', 'Debut_Week_3', 'Debut_Week', 'Debut_Position', 'Debut_Position_3'], inplace = True)

songs_sub.rename(columns={'Debut_Week_2':'Debut_Week',
                          'Debut_Position_2':'Debut_Position'},
                 inplace=True)

#plot weeks on chart to check Weibull distribution
fig = plt.figure(figsize = (15,5))
ax = fig.add_subplot(1,1,1)
ax = songs_sub['Weeks on Chart'].plot.hist(bins = 50)
plt.show()

#right censoring, get rid of any weeks that have a last flag at week28 (12-28-2019)
print(songs_sub.info())
songs_sub = songs_sub.sort_values(by = 'WeekID')
songs_sub = songs_sub.reset_index(drop = True)

week28 = songs_sub.at[30797, 'WeekID']
'''Testing: print(week28)'''
songs_sub_2 = songs_sub.loc[songs_sub.WeekID != week28, :]
print(songs_sub_2.info())

#add Month and year for Debut week for controls later
songs_sub_2.Debut_Week = pd.to_datetime(songs_sub_2.Debut_Week)
songs_sub_2['Month'] = songs_sub_2['Debut_Week'].dt.strftime('%m')
songs_sub_2['Year'] = songs_sub_2['Debut_Week'].dt.strftime('%Y')
print(songs_sub_2.info())

#famous variable - debut position should control for fame/marketing?
#songs_sub_2.sort_values(by = ['Performer', 'Song'])

songs_sub_2.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'], inplace = True)
#print(songs_sub_2.info())
songs_sub_2.to_csv('songs_cleanish_5.csv')

Cleaning part 4: merging datasets, cleaning up extraneous variables for import into Stata and SQL

In [ ]:
songs = pd.read_csv('songs_cleanish_5.csv')
keywords_sent = pd.read_csv('lyric-keywords-sent.csv')
aud_features = pd.read_excel('Hot 100 Audio Features.xlsx')
lyrics = pd.read_csv('lyric-full_df.csv')

#import unemployment data and change to ints
unemployment = pd.read_csv('UNRATE.csv')
unemployment.DATE = pd.to_datetime(unemployment.DATE)
unemployment['Month'] = unemployment['DATE'].dt.strftime('%m').astype(int)
unemployment['Year'] = unemployment['DATE'].dt.strftime('%Y').astype(int)

print(songs.info())
print(lyrics.info())
print(keywords_sent)
print(aud_features.info())

songs.WeekID = pd.to_datetime(songs.WeekID)

#chose to implement later/in stata
''''Indicator variables after 2000: recession and unemployment
cond1 = songs['WeekID'] >= datetime(2007, 12, 1)
cond2 = songs['WeekID'] <= datetime(2009, 5, 1)
cond3 = songs['WeekID'] >= datetime(2009, 4, 1)
cond4 = songs['WeekID'] <= datetime(2011, 8, 1)
songs['Recession'] = np.where(cond1 & cond2, 1, 0)
songs['Unemploy'] = np.where(cond3 & cond4, 1, 0)
'''

#merge with unemployment data
songs_unemploy = pd.merge(songs, unemployment[['Month','Year', 'UNRATE']],
                          on = ['Month', 'Year'],
                          how = 'left',
                          left_index=True)

#merge with sentiment analysis, kept lyrics just in case for further NPL
songs_lyrics = pd.merge(songs_unemploy,
                        lyrics[['SongID','Lyrics', 'Sentiment']],
                        on = 'SongID',
                        how = 'left',
                        left_index = True)

#merge with Spotify audio features
songs_all = pd.merge(songs_lyrics,
                     aud_features[['SongID','spotify_genre','spotify_track_duration_ms','mode', 'valence', 'tempo',
                                   'danceability','energy','speechiness', 'acousticness','loudness', 'key']],
                     on = 'SongID',
                     how = 'left',
                     left_index = True)

#clean up
songs_all.drop(columns='Unnamed: 0', inplace=True)
print(songs_all.info())
songs_all = songs_all.drop_duplicates(keep='first')
print(songs_all.info())

#plot and check distribution
songs_all.sort_values(by = 'Weeks on Chart')
fig = plt.figure(figsize = (15,5))
ax = fig.add_subplot(1,1,1)
songs_all['survival'] = np.log(songs_all['Weeks on Chart'])
ax = songs_all['survival'].plot.hist(bins = 50)
ax = songs_all['Weeks on Chart'].plot.hist(bins = 50)
plt.show()

#clean up for import into stata and SQL (no strings with commas)
songs_all.drop(columns=['Lyrics', 'spotify_genre'], inplace=True)
songs_all = songs_all.reset_index(drop = True)

print(songs_all.size)
songs_all.to_csv('songs_all_2.csv')

#notes:
#lyrics goes up to 2019-06-22
#songs_all_2 = songs_all.reset_index().drop_duplicates(subset='index',
                                 #      keep='first').set_index('index')
'''Testing: 
nullsongs = songs_all_2[songs_all_2.isna().any()]
print(nullsongs.info())'''

In [ ]:
songs = pd.read_csv('songs_all_2.csv')
print(songs.shape)
songs = songs.sort_values(by = ['SongID', 'Instance'])
songs = songs.reset_index(drop = True)

#Weeks substracted for instances
cond1 = songs['Instance'] > songs['Instance'].shift()
songs['WeekChart2'] = np.where((cond1),
            songs['Weeks on Chart']-songs['Weeks on Chart'].shift(),
            songs['Weeks on Chart'])
'''Testing: songs.survival = np.log(songs.WeekChart2)
songs['WeekChart2'].plot.hist(bins = 100)
songs['survival'].plot.hist(bins = 50)
plt.show()'''

#setting recession indicator
songs.Debut_Week = pd.to_datetime((songs.Debut_Week))
#print(songs.info())
cond1 = (songs.Debut_Week > ('1960-04-01')) & (songs.Debut_Week < ('1961-02-01'))
cond2 = (songs.Debut_Week > ('1970-01-01')) & (songs.Debut_Week < ('1970-11-01'))
cond3 = (songs.Debut_Week > datetime(1973,12,1)) & (songs.Debut_Week < datetime(1975, 2,1))
cond4 = (songs.Debut_Week > datetime(1980,1,1)) & (songs.Debut_Week < datetime(1980, 5,1))
cond5 = (songs.Debut_Week > datetime(1981,8,1)) & (songs.Debut_Week < datetime(1982, 11,1))
cond6 = (songs.Debut_Week > datetime(1990,8,1)) & (songs.Debut_Week < datetime(1991, 3,1))
mask = (cond1 | cond2 | cond3 | cond4 | cond5 | cond6)
songs['recession'] = np.where(mask, 1, 0)

#export
songs.to_csv('songs_all_3_2.csv')

#restrict sample to before 1992 only due to week20 abnormality after 1992
songs_temp = songs.loc[songs.WeekID < datetime(1992, 1, 1), :]
print(songs_temp)
songs_temp.to_csv('songs_before_1992.csv')

'''Testing: print(songs.loc[songs.Song =='22', :])
songs_temp.WeekChart2.plot.hist(bins = 80)
plt.show()
songs.WeekChart2.plot.hist(bins = 100)
plt.show()
'''

#taking out instances
songs = pd.read_csv('songs_all_2.csv')
songs = songs.sort_values(by = ['SongID', 'Instance'])
songs = songs.reset_index(drop = True)

cond2 = songs['Peak Position'] > songs['Peak Position'].shift()
cond3 = songs['Song'] == songs['Song'].shift()

songs['Peak'] = np.where((cond2) & (cond3),
                         songs['Peak Position'].shift(),
                         songs['Peak Position'])

print(songs.shape)
songs = songs.sort_values(by = ['SongID', 'Instance'])
songs.flag_last = ((songs.SongID != songs.SongID.shift(-1))).astype(int)
songs = songs.loc[songs.flag_last == 1, :]

#export various
songs.to_csv('songs_all_without_instances.csv')

songs.WeekID = pd.to_datetime(songs.WeekID)
songs_before = songs.loc[songs.WeekID < datetime(1992, 1, 1), :]
songs_before.to_csv('songs_before_1992 without_instances.csv')

songs_before = songs_before.loc[songs['Peak'] <= 10, :]
songs_before.to_csv('top10_songs_before_1992_without_instances.csv')